In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.metrics import make_scorer
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
import joblib
import pretty_midi
import numpy as np
from scipy.stats import entropy
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from statistics import mean
from os import listdir
from os.path import getsize

In [3]:
file = joblib.load('../raw_data/pretty_midi/(Day Dream) Prayer')
file2 = pretty_midi.PrettyMIDI('../raw_data/1.mid')

In [4]:
def pattern_recognition(acc, mel):
    """
    Create a pattern of the evolution of relative pitch and rythm from accompaniment and melody
    """
    # Create list of pitches for each frame in accompaniment's piano roll
    acc_pitches = []
    
    for frame in acc.T:
        pitches = []
        for vel in frame:
            if vel > 0:
                pitches.append(list(frame).index(vel))
        if not pitches:
            pitches.append(0)
        acc_pitches.append(pitches)
    
    # Take the mean pitch of each frame
    mean_pitches_acc = [(np.sum(pitches) / len(pitches)) for pitches in acc_pitches]
    
    # Get only the pitches (not the velocities)
    melody_pitches = mel[:500]
    
    
    # Create the pattern for accompaniment
    acc_passed_pitch = []
    pitch_pattern_acc = [0]
    
    # Do we start with a note played or not ?
    if mean_pitches_acc[0] > 0:
        rythm_pattern_acc = [1]
    else:
        rythm_pattern_acc = [0]
        
    for pitch in mean_pitches_acc:
        if acc_passed_pitch:
            if pitch != acc_passed_pitch[-1]:
                relative_pitch = pitch - acc_passed_pitch[-1]
                pitch_pattern_acc.append(relative_pitch)
                rythm_pattern_acc.append(1)
            else:
                pitch_pattern_acc.append(pitch_pattern_acc[-1])
                rythm_pattern_acc.append(0)
        acc_passed_pitch.append(pitch)
    
    # Create the pattern for melody
    mel_passed_pitch = []
    pitch_pattern_mel = [0]
    
    # Do we start with a note played or not ?
    if melody_pitches[0] > 0:
        rythm_pattern_mel = [1]
    else:
        rythm_pattern_mel = [0]
        
    for pitch in melody_pitches:
        if mel_passed_pitch:
            if pitch != mel_passed_pitch[-1]:
                relative_pitch = pitch - mel_passed_pitch[-1]
                pitch_pattern_mel.append(relative_pitch)
                rythm_pattern_mel.append(1)
            else:
                pitch_pattern_mel.append(pitch_pattern_mel[-1])
                rythm_pattern_mel.append(0)
        mel_passed_pitch.append(pitch)

    return {
        'rythm_pattern':{'acc':rythm_pattern_mel,'mel':rythm_pattern_mel},
        'pitch_pattern':{'acc':pitch_pattern_mel,'mel':pitch_pattern_mel}
           }    
    
def custom_metric(accompaniment, predicted_melody, weight=1):
    """
    TODO : Doc string
    """
    # Get the pitch and rythm pattern for both accompaniment and melody
    patterns = pattern_recognition(accompaniment, predicted_melody)
    pitch_pattern_acc = patterns['pitch_pattern']['acc']
    pitch_pattern_mel = patterns['pitch_pattern']['mel']
    rythm_pattern_acc = patterns['rythm_pattern']['acc']
    rythm_pattern_mel = patterns['rythm_pattern']['mel']
    
    # Compute the mean of velocities for both accompaniment and melody
    list_mean_vel_acc = [
        np.mean(
            [vel for vel in frame if vel > 0]
        )
        for frame in accompaniment.T
        if np.sum(frame) > 0]
    
    # Check if the list is empty
    if not list_mean_vel_acc:
        list_mean_vel_acc = [0]
    
    mean_vel_acc = np.mean(list_mean_vel_acc)
    
    list_mean_vel_pred = [pred for pred in predicted_melody[500:] if pred > 0]
    
    # Check if the list is empty
    if not list_mean_vel_pred:
        list_mean_vel_pred = [0]
        
    mean_vel_pred = np.mean(list_mean_vel_pred)
    
    # Compute the diff beetween the two velocities mean
    velocity_diff = abs(mean_vel_acc - mean_vel_pred)
    
    # Compute the "diff pattern" beetween accompaniment and melody
    diff_pitch_pattern = np.array([abs(acc - mel) for acc, mel in zip(pitch_pattern_acc, pitch_pattern_mel)]).reshape(-1, 1)
    
    # Compute the entropy of the diff pattern
    if np.sum(diff_pitch_pattern) == 0:
        pitch_entropy_score = 0
    else:
        pitch_entropy_score = entropy(diff_pitch_pattern)[0]
        
    # Now we do the same for rythm pattern
    diff_rythm_pattern = np.array([acc + mel for acc, mel in zip(rythm_pattern_acc, rythm_pattern_mel)]).reshape(-1, 1)
        
    # Compute the entropy of the diff pattern
    if np.sum(diff_rythm_pattern) == 0:
        rythm_entropy_score = 0
    else:
        rythm_entropy_score = entropy(diff_rythm_pattern)[0]
        
    # Compute the "note's length score" of the melody
    notes_length_score = rythm_pattern_mel.count(1) * weight
    
    # Compute the final score
    entropy_score = pitch_entropy_score + rythm_entropy_score

    return np.mean([entropy_score, velocity_diff, notes_length_score])

### Compute the score within a grid search

In [5]:
grid = {
    'criterion':               ["squared_error","friedman_mse","absolute_error","poisson"],
    'max_depth':               [None, 2, 12, 128],
    'min_samples_split':       [2, 3, 5, 10],
    'min_samples_leaf':        [1, 2, 3, 4],
    'min_weight_fraction_leaf':[0.0, 0.2, 0.4, 0.5],
    'max_leaf_nodes':          [None, 128, 12, 2],
    'min_impurity_decrease':   [0.0, 0.2, 0.5, 0.8],
}

In [6]:
# First grid search best params

{'params': {'criterion': 'friedman_mse',
  'max_depth': 128,
  'min_samples_split': 2},
 'score': 8.528335277288813}

# Second grid search best params

{'params': {'criterion': 'friedman_mse',
  'max_depth': 128,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128},
 'score': 9.93505034874958}

# Last grid search best params

{'params': {'criterion': 'friedman_mse',
  'max_depth': 128,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128,
  'min_impurity_decrease': 0.2},
 'score': 6.957092800922064}

{'params': {'criterion': 'friedman_mse',
  'max_depth': 128,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128,
  'min_impurity_decrease': 0.2},
 'score': 6.957092800922064}

In [7]:
{'params': {'criterion': 'friedman_mse',
  'max_depth': 128,
  'min_samples_split': 2},
 'score': 6.755850017125949}

{'params': {'criterion': 'friedman_mse',
  'max_depth': 128,
  'min_samples_split': 2},
 'score': 6.755850017125949}

In [27]:
params = [{
    'criterion':crit,
           'max_depth':128,
           'min_samples_split':2,
           'min_samples_leaf':1,
           'min_weight_fraction_leaf':0.0,
           'max_leaf_nodes':128,
           'min_impurity_decrease':0.5
}
          for crit in grid['criterion']
          # for max_d in grid['max_depth']
          # for min_ss in grid['min_samples_split']
          # # for min_sl in grid['min_samples_leaf']
          # # for min_w in grid['min_weight_fraction_leaf']
          # # for max_l in grid['max_leaf_nodes']
          # for min_i in grid['min_impurity_decrease']
]

In [28]:
len(params)

4

In [10]:
path = '../raw_data/pretty_midi'
directory = [file_name for file_name in listdir(path) if getsize(f'{path}/{file_name}') < 300_000]

for i, file_name in enumerate(directory):
    file = joblib.load(f'{path}/{file_name}')
    if i == 0:
        X, y = create_simple_dataset(file, ratio=0.2)
    else:
        try:
            loaded = create_simple_dataset(file, ratio=0.2)
            X = np.concatenate((X, loaded[0]))
            y = np.concatenate((y, loaded[1]))
            del([loaded, file])
        except:
            pass
    if i % 10 == 0:
        print(f'{X.shape[0]} observations')
    if X.shape[0] >= 0:
        break

chord = adding_chords_info('../raw_data/chords_midi.csv', X)

X_reshaped = X.reshape((X.shape[0], -1))
X_reshaped = np.concatenate((chord, X_reshaped), axis=1, dtype=np.int8)

y = y.reshape((y.shape[0], -1))

14 observations


In [29]:
params_and_scores = {}

for split in range(5):
    X_train, X_test, X_reshaped_train, X_reshaped_test, y_train, y_test = train_test_split(X, X_reshaped, y, test_size=0.1)
    params_and_scores[f'split_{split}'] = []
    for i, param in enumerate(params):

        tree = DecisionTreeRegressor(**param)
        tree.fit(X_reshaped_train, y_train)
        predictions = tree.predict(X_reshaped_test)
        scores = [custom_metric(test, pred, weight=10) for test, pred in zip(X_test, predictions)]
        score = np.mean(scores)
        params_and_scores[f'split_{split}'].append({'params':param, 'score':score})
    print(f'split {split+1} done')

split 1 done
split 2 done
split 3 done
split 4 done
split 5 done


In [30]:
scores = {}
for split in params_and_scores:
    scores[split] = []
    for param in params_and_scores[split]:
        scores[split].append(param['score'])

In [31]:
best_scores = {}
for split in scores:
    best_scores[split] = np.min(scores[split])

In [32]:
scores_per_params = {}

for split in scores:
    
    # Create the lists
    for index, score in enumerate(scores[split]):
        scores_per_params[index] = []
    
for split in scores:
    
    # Fill the lists
    for index, score in enumerate(scores[split]):
        scores_per_params[index].append(score)

mean_scores_per_params = [(key, np.mean(score)) for key, score in scores_per_params.items()]

In [33]:
min_score = np.min([s[1] for s in mean_scores_per_params])
best_params = params_and_scores['split_0'][[s[1] for s in mean_scores_per_params].index(min_score)]

In [34]:
best_params

{'params': {'criterion': 'absolute_error',
  'max_depth': 128,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128,
  'min_impurity_decrease': 0.5},
 'score': 58.05813945590767}

#### Best params for each grid search, metrics being updated each time

In [ ]:
# Best params last grid search, with note's length

{'params': {'criterion': 'absolute_error',
  'max_depth': 128,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128,
  'min_impurity_decrease': 0.5},
 'score': 58.05813945590767}

In [17]:
# best params second grid search, without note's length

best_params = {'params': {'criterion': 'friedman_mse',
  'max_depth': None,
  'min_samples_split': 3,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128,
  'min_impurity_decrease': 0.8},
 'score': 3.934009184910246}

# best params first grid search
{'params': {'max_depth': None,
  'min_samples_split': 3,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 128},
 'score': 4.371726592684219}

# worst params first grid search
{'params': {'max_depth': None,
  'min_samples_split': 3,
  'min_samples_leaf': 4,
  'min_weight_fraction_leaf': 0.2,
  'max_leaf_nodes': None},
 'score': 11.102578913240704}

{'params': {'max_depth': None,
  'min_samples_split': 3,
  'min_samples_leaf': 4,
  'min_weight_fraction_leaf': 0.2,
  'max_leaf_nodes': None},
 'score': 11.102578913240704}

In [18]:
# Best params first grid search, without rythm pattern and note's length, without CV

{'params': {'criterion': 'absolute_error',
  'max_depth': 12,
  'min_samples_split': 5},
 'score': 3.9304437664246548}

# Best params second grid search with CV

{'params': {'criterion': 'absolute_error',
  'max_depth': 12,
  'min_samples_split': 5,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 12},
 'score': 2.427159698274215}

# Best params last grid search with CV

{'params': {'criterion': 'absolute_error',
  'max_depth': 12,
  'min_samples_split': 5,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 12,
  'min_impurity_decrease': 0.5},
 'score': 3.376318707323766}

{'params': {'criterion': 'absolute_error',
  'max_depth': 12,
  'min_samples_split': 5,
  'min_samples_leaf': 1,
  'min_weight_fraction_leaf': 0.0,
  'max_leaf_nodes': 12,
  'min_impurity_decrease': 0.5},
 'score': 3.376318707323766}